In [6]:
"""
Align 5 forecast files to the same dates and compute a Diebold–Mariano matrix.

Outputs (saved to the current directory):
- date_alignment_summary.csv
- aligned_forecasts.csv
- dm_statistics.csv
- dm_p_values.csv
"""

import pandas as pd
import numpy as np
from itertools import combinations
from math import erf, sqrt

# -----------------------------
# 1) File paths + column names
# -----------------------------
FILES = {
    'ARIMA': {'path': 'arima_forecast_output.csv',       'forecast_col': 'forecast',             'actual_col': 'actual'},
    'GRU':   {'path': 'gru_forecast_output.csv',         'forecast_col': 'predicted',            'actual_col': 'actual'},
    'RF':    {'path': 'random_forest_predictions.csv',   'forecast_col': 'prediction',           'actual_col': 'actual'},
    'RW':    {'path': 'random_walk_forecast_output.csv', 'forecast_col': 'random_walk_forecast', 'actual_col': 'value'},
    'SVR':   {'path': 'svr_forecast_output.csv',         'forecast_col': 'predicted',            'actual_col': 'actual'},
}

# ---------------------------------
# 2) Helper functions for date fix
# ---------------------------------
def to_datetime_norm(s):
    """Parse dates robustly -> timezone-naive midnight (YYYY-MM-DD 00:00:00)."""
    dt = pd.to_datetime(s, errors='coerce', infer_datetime_format=True)
    try:
        dt = dt.dt.tz_localize(None)
    except Exception:
        try:
            dt = dt.dt.tz_convert(None)
        except Exception:
            pass
    return dt.dt.normalize()

def is_index_like_date_column(raw_series, parsed_dt):
    """
    Heuristics to flag 'date' that are actually indices (e.g., integers) or invalid:
    - Original dtype numeric OR
    - Parsed years all <= 1971 (near epoch) OR
    - Parsed all the same day OR
    - Mostly NaT after parsing
    """
    if pd.api.types.is_numeric_dtype(raw_series):
        return True
    if parsed_dt.isna().mean() > 0.5:  # >50% failed parse
        return True
    years = parsed_dt.dropna().dt.year
    if len(years) == 0:
        return True
    if years.min() <= 1971 and years.max() <= 1971:
        return True
    if parsed_dt.nunique(dropna=True) == 1:
        return True
    return False

def normal_sf(z):
    """Survival function 1 - Phi(|z|) using error function, no SciPy needed."""
    return 0.5 * (1 - erf(abs(z) / sqrt(2)))

# -----------------------------------------
# 3) Load files, standardize, inspect dates
# -----------------------------------------
raw = {}
before_rows = []
for model, info in FILES.items():
    df = pd.read_csv(info['path'])
    # standardize columns
    df = df.rename(columns={info['forecast_col']: 'forecast', info['actual_col']: 'actual'})
    if 'date' not in df.columns:
        raise ValueError(f"{model}: missing 'date' column.")
    df['date_raw'] = df['date']  # keep for diagnostics
    dt = to_datetime_norm(df['date'])
    idx_like = is_index_like_date_column(df['date'], dt)

    raw[model] = {
        'df': df[['date', 'date_raw', 'actual', 'forecast']].copy(),
        'dt': dt,
        'index_like': idx_like
    }
    before_rows.append({
        'model': model,
        'original_start': dt.min(),
        'original_end': dt.max(),
        'rows': len(df),
        'index_like_date': idx_like
    })

before_df = pd.DataFrame(before_rows).sort_values('model').reset_index(drop=True)
print("== Original date diagnostics ==")
print(before_df)

# -----------------------------------------------------------
# 4) Choose a reference calendar among valid-dated dataframes
# -----------------------------------------------------------
candidates = [(m, info['dt'], info['index_like'], len(info['df'])) for m, info in raw.items()]
valid = [(m, dt, n) for (m, dt, ix, n) in candidates if not ix and dt.notna().any()]
if not valid:
    raise ValueError("No file with valid real dates to use as a reference calendar.")

# Pick the valid model with the most rows as reference
ref_model, ref_dt, ref_n = sorted(valid, key=lambda x: (-x[2], x[1].min()))[0]
ref_calendar = pd.Series(ref_dt.dropna().sort_values().unique())
print(f"\nUsing '{ref_model}' as reference calendar: {ref_calendar.iloc[0].date()} → {ref_calendar.iloc[-1].date()} "
      f"({len(ref_calendar)} unique dates)")

# ---------------------------------------------------------
# 5) Build fixed dataframes with proper 'date' column set
# ---------------------------------------------------------
fixed = {}
issues = []
for m, info in raw.items():
    df = info['df'].copy()
    if info['index_like']:
        # Try to borrow the reference calendar by order if lengths match
        if len(df) == len(ref_calendar):
            df['date'] = ref_calendar.values
        else:
            issues.append((m, len(df), len(ref_calendar)))
            # Fallback: keep parsed (likely invalid) dates to fail later gracefully
            df['date'] = to_datetime_norm(df['date'])
    else:
        df['date'] = info['dt']  # already parsed valid dates

    # Keep only necessary columns and sort
    df = df[['date', 'actual', 'forecast']].dropna(subset=['date']).sort_values('date').reset_index(drop=True)
    fixed[m] = df

if issues:
    raise ValueError(
        "Cannot fix index-like dates for these files because their lengths don't match the reference calendar:\n"
        + "\n".join([f" - {m}: len={n} vs ref={r}" for (m, n, r) in issues])
    )

# -------------------------------------------------------
# 6) Align all to the exact intersection of available dates
# -------------------------------------------------------
date_sets = [set(df['date']) for df in fixed.values()]
common_dates = sorted(set.intersection(*date_sets))
if not common_dates:
    raise ValueError("No overlapping dates across all models after fixing dates.")

# Trim each DF to the common calendar
for m in fixed:
    fixed[m] = fixed[m][fixed[m]['date'].isin(common_dates)].sort_values('date').reset_index(drop=True)

after_rows = []
for m, df in fixed.items():
    after_rows.append({
        'model': m, 'aligned_start': df['date'].min(),
        'aligned_end': df['date'].max(), 'rows': len(df)
    })
after_df = pd.DataFrame(after_rows).sort_values('model').reset_index(drop=True)
print("\n== Aligned date diagnostics ==")
print(after_df)

# ---------------------------------------------
# 7) Build combined panel for actual & forecasts
# ---------------------------------------------
combined = pd.DataFrame({'date': pd.Series(common_dates)})
# Merge each model's actual & forecast; keep per-model actuals for robust combining
for m, df in fixed.items():
    tmp = df[['date', 'actual', 'forecast']].copy()
    tmp = tmp.rename(columns={'actual': f'actual_{m}', 'forecast': m})
    combined = combined.merge(tmp, on='date', how='inner')

# If all 'actual_*' equal, just take one; else take row-wise median (robust)
actual_cols = [c for c in combined.columns if c.startswith('actual_')]
if len(actual_cols) == 0:
    raise ValueError("No 'actual' columns found after merge.")
equal_actuals = all(combined[actual_cols[0]].equals(combined[c]) for c in actual_cols[1:])
if equal_actuals:
    combined['actual'] = combined[actual_cols[0]].values
else:
    combined['actual'] = np.nanmedian(combined[actual_cols].values, axis=1)

# Optional: drop per-model actuals to keep the table tidy
# combined = combined.drop(columns=actual_cols)

# ---------------------------------------------------------
# 8) Diebold–Mariano test (squared-error loss) + HAC (NW)
# ---------------------------------------------------------
def dm_test(e1, e2, hac_lags=None):
    """
    e1, e2: np.array forecast errors (actual - forecast), same length
    hac_lags: Newey–West bandwidth; default ~ 1.5*T^(1/3)
    Returns: (DM_stat, two-sided p-value)
    """
    e1 = np.asarray(e1, float)
    e2 = np.asarray(e2, float)
    if len(e1) != len(e2):
        raise ValueError("Errors must have same length.")

    d = (e1**2 - e2**2)
    T = len(d)
    d_bar = d.mean()

    if hac_lags is None:
        hac_lags = int(np.floor(1.5 * (T ** (1/3))))  # automatic bandwidth

    # Newey–West HAC variance estimate of d
    gamma0 = np.var(d, ddof=1)
    var_hat = gamma0
    for k in range(1, min(hac_lags, T-1) + 1):
        w = 1.0 - k / (hac_lags + 1.0)  # Bartlett weight
        cov = np.cov(d[k:], d[:-k], bias=True)[0, 1]
        var_hat += 2.0 * w * cov

    var_dbar = var_hat / T if T > 0 else np.nan
    if not np.isfinite(var_dbar) or var_dbar <= 0:
        return 0.0, 1.0  # fallback

    DM = d_bar / np.sqrt(var_dbar)
    # two-sided p; Phi via erf
    pval = 2 * normal_sf(DM)
    return DM, pval

models = list(fixed.keys())
dm_stats = pd.DataFrame(0.0, index=models, columns=models)
dm_pvals = pd.DataFrame(0.0, index=models, columns=models)

errors = {m: (combined['actual'].values - combined[m].values).astype(float) for m in models}
for m1, m2 in combinations(models, 2):
    stat, pval = dm_test(errors[m1], errors[m2])
    dm_stats.loc[m1, m2] = stat
    dm_stats.loc[m2, m1] = -stat  # antisymmetry of d(e1^2 - e2^2)
    dm_pvals.loc[m1, m2] = pval
    dm_pvals.loc[m2, m1] = pval   # symmetric p-values

# Diagonals
np.fill_diagonal(dm_stats.values, 0.0)
np.fill_diagonal(dm_pvals.values, 0.0)

# -----------------------
# 9) Save + print result
# -----------------------
before_df.to_csv('date_alignment_summary.csv', index=False)
combined.to_csv('aligned_forecasts.csv', index=False)
dm_stats.to_csv('dm_statistics.csv')
dm_pvals.to_csv('dm_p_values.csv')

print("\n== Diebold–Mariano Test Statistics ==")
print(dm_stats.round(4))
print("\n== Diebold–Mariano P-Values ==")
print(dm_pvals.round(4))

print("\nSaved files:")
print(" - date_alignment_summary.csv")
print(" - aligned_forecasts.csv")
print(" - dm_statistics.csv")
print(" - dm_p_values.csv")


== Original date diagnostics ==
   model original_start original_end  rows  index_like_date
0  ARIMA     1970-01-01   1970-01-01  1344             True
1    GRU     2020-01-02   2025-03-27  1344            False
2     RF     2020-01-03   2025-03-27  1343            False
3     RW     2020-01-02   2025-03-27  1344            False
4    SVR     2020-01-02   2025-03-27  1344            False

Using 'GRU' as reference calendar: 2020-01-02 → 2025-03-27 (1344 unique dates)

== Aligned date diagnostics ==
   model aligned_start aligned_end  rows
0  ARIMA    2020-01-03  2025-03-27  1343
1    GRU    2020-01-03  2025-03-27  1343
2     RF    2020-01-03  2025-03-27  1343
3     RW    2020-01-03  2025-03-27  1343
4    SVR    2020-01-03  2025-03-27  1343

== Diebold–Mariano Test Statistics ==
        ARIMA     GRU      RF      RW     SVR
ARIMA  0.0000  7.2780  7.2650  7.2793  7.2796
GRU   -7.2780  0.0000 -6.1098  2.8019  2.6941
RF    -7.2650  6.1098  0.0000  6.4312  6.4683
RW    -7.2793 -2.8019 -6.43

C:\Users\alius\AppData\Local\Temp\ipykernel_20756\1883472139.py:32: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  dt = pd.to_datetime(s, errors='coerce', infer_datetime_format=True)
C:\Users\alius\AppData\Local\Temp\ipykernel_20756\1883472139.py:32: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  dt = pd.to_datetime(s, errors='coerce', infer_datetime_format=True)
C:\Users\alius\AppData\Local\Temp\ipykernel_20756\1883472139.py:32: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is